# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install ..

Found existing installation: todd-ai 0.3.0
Uninstalling todd-ai-0.3.0:
  Successfully uninstalled todd-ai-0.3.0
Note: you may need to restart the kernel to use updated packages.
Processing /Users/bytedance/Developer/todd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for todd-ai: filename=todd_ai-0.3.0-py3-none-any.whl size=97392 sha256=2ef8e0b33cb10c9328bc34c82452773af7ae42c391521feac502f4ff422bcb71
  Stored in directory: /private/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/pip-ephem-wheel-cache-g56xo30x/wheels/15/ef/5a/9fc12e257ce5cef16b333a2ed6c992ff9cbcc9167f7199e6ac
Successfully built todd-ai

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import tempfile
from typing import Any, TypedDict, cast

import torch
import torch.nn.functional as F
import torch.utils.data

import todd

Memo = dict[str, Any]

/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
[2023-07-18 16:09:50,137 61042:140704352601664][patches.py:72 todd <module>] INFO: `ipdb` is installed. Using it for debugging.


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register()
class RunnerModel(todd.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int


In [5]:
@todd.DatasetRegistry.register()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

### Runners

In [7]:
class RunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, batch: Batch, memo: Memo) -> None:
        y: torch.Tensor = self._strategy.model(batch['x'])
        loss = F.l1_loss(y, batch['y'])
        memo['loss'] = loss
        if 'log' in memo:
            memo['log']['loss'] = f'{loss.item():.3f}'

In [8]:
class TrainerMixin(RunnerMixin):

    def _run_iter(self, batch: Batch, memo: Memo) -> None:
        super()._run_iter(batch, memo)
        if 'log' in memo:
            model = cast(RunnerModel, self._strategy.model)
            memo['log']['weight'] = f'{model.weight.item():.3f}'
            memo['log']['batch'] = str(batch)

In [9]:
@todd.RunnerRegistry.register()
class CustomValidator(RunnerMixin, todd.runners.Validator):
    pass

In [10]:
@todd.RunnerRegistry.register()
class CustomIterBasedTrainer(TrainerMixin, todd.runners.IterBasedTrainer):
    pass

In [11]:
@todd.RunnerRegistry.register()
class CustomEpochBasedTrainer(TrainerMixin, todd.runners.EpochBasedTrainer):
    pass

## Validation

In [12]:
validator_demo = todd.Config(
    type='CustomValidator',
    name='custom_validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='VanillaStrategy', model=dict(type='RunnerModel')),
    callbacks=dict(type='LogCallback', interval=5),
)

In [13]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-07-18 16:09:50,762 61042:140704352601664][log.py:37 todd.CustomValidator.custom_validator connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:50,765 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-18 16:09:50,767 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-18 16:09:50,769 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-18 16:09:50,770 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpd_rerfmh
└── custom_validator

2 directories, 0 files


## Train

In [14]:
trainer_demo = validator_demo.copy()
trainer_demo.pop('type')
trainer_demo.dataloader = todd.Config(
    batch_size=2, shuffle=True, dataset=dict(type='RunnerDataset', n=67),
)
trainer_demo.optimizer = todd.Config(type='SGD', lr=0.005)


### Iteration Based

In [15]:
iter_based_trainer_demo = trainer_demo.copy()
iter_based_trainer_demo.type = 'CustomIterBasedTrainer'
iter_based_trainer_demo.name = 'custom_iter_based_trainer'
iter_based_trainer_demo.iters = 53

In [16]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        iter_based_trainer_demo,
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

[2023-07-18 16:09:51,067 61042:140704352601664][log.py:37 todd.CustomIterBasedTrainer.custom_iter_based_trainer connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:51,072 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=77.000 weight=0.000 batch={'x': tensor([32, 45]), 'y': tensor([64, 90])}
[2023-07-18 16:09:51,074 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=88.000 weight=0.000 batch={'x': tensor([66, 22]), 'y': tensor([132,  44])}
[2023-07-18 16:09:51,076 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=62.000 weight=0.000 batch={'x': tensor([46, 16]), 'y': tensor([92, 32])}
[2023-07-18 16:09:51,078 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [20/53] loss

### Epoch Based

In [17]:
epoch_based_trainer_demo = trainer_demo.copy()
epoch_based_trainer_demo.type = 'CustomEpochBasedTrainer'
epoch_based_trainer_demo.name = 'custom_epoch_based_trainer'
epoch_based_trainer_demo.epochs = 3

In [18]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        epoch_based_trainer_demo,
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

[2023-07-18 16:09:51,179 61042:140704352601664][log.py:37 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:51,180 61042:140704352601664][log.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-18 16:09:51,183 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=101.000 weight=0.000 batch={'x': tensor([52, 49]), 'y': tensor([104,  98])}
[2023-07-18 16:09:51,186 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=123.000 weight=0.000 batch={'x': tensor([60, 63]), 'y': tensor([120, 126])}
[2023-07-18 16:09:51,188 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=37.000 weight=0.000 batch={'x': tensor([18, 19]), 'y': tens

## Callbacks

### Log

In [19]:
log_callback_demo = validator_demo.copy()
log_callback = log_callback_demo.callbacks
log_callback.with_file_handler = True
log_callback_demo.callbacks = [log_callback]

In [20]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        log_callback_demo, 
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-07-18 16:09:51,243 61042:140704352601664][log.py:37 todd.CustomValidator.custom_validator connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:51,245 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-18 16:09:51,247 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-18 16:09:51,249 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-18 16:09:51,251 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpes7i_xfn
└── custom_validator
    └── 2023-07-18T16-09-51_242721-08-00.log

2 directories, 1 file

[2023-07-18 16:09:51,243 61042:140704352601664][log.py:37 todd.CustomValidator.custom_validator connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:51,245 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-18 16:09:51,247 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-18 16:09:51,249 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-18 16:09:51,251 61042:140704352601664][log.py:52 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000


### Optimize

In [21]:
optimize_callback_demo = iter_based_trainer_demo.copy()
optimize_callback = todd.Config(type='OptimizeCallback')
optimize_callback_demo.callbacks = [optimize_callback, log_callback]

In [22]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        optimize_callback_demo,
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

[2023-07-18 16:09:51,835 61042:140704352601664][log.py:37 todd.CustomIterBasedTrainer.custom_iter_based_trainer connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:51,841 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=74.231 weight=0.612 batch={'x': tensor([62, 45]), 'y': tensor([124,  90])}
[2023-07-18 16:09:51,845 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=23.500 weight=1.412 batch={'x': tensor([49, 31]), 'y': tensor([98, 62])}
[2023-07-18 16:09:51,848 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=1.635 weight=2.030 batch={'x': tensor([63, 46]), 'y': tensor([126,  92])}
[2023-07-18 16:09:51,852 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [20/53] los

### Learning Rate Schedule

In [23]:
lr_schedule_callback_demo = iter_based_trainer_demo.copy()
lr_schedule_callback = todd.Config(
    type='LRScheduleCallback',
    lr_scheduler=dict(type='LinearLR', total_iters=10),
)
lr_schedule_callback_demo.callbacks = [optimize_callback, lr_schedule_callback, log_callback,]


In [24]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_callback_demo,
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

[2023-07-18 16:09:51,894 61042:140704352601664][log.py:37 todd.CustomIterBasedTrainer.custom_iter_based_trainer connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:51,898 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=51.566 weight=0.363 batch={'x': tensor([60,  3]), 'y': tensor([120,   6])} lr=['3.333e-03']
[2023-07-18 16:09:51,902 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=25.368 weight=1.005 batch={'x': tensor([37, 14]), 'y': tensor([74, 28])} lr=['5.000e-03']
[2023-07-18 16:09:51,905 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=5.879 weight=1.862 batch={'x': tensor([53, 32]), 'y': tensor([106,  64])} lr=['5.000e-03']
[2023-07-18 16:09:51,909 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_b

In [25]:
lr_schedule_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
lr_schedule_by_epoch_callback = lr_schedule_callback.copy()
lr_schedule_by_epoch_callback.by_epoch = True
lr_schedule_by_epoch_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_by_epoch_callback, log_callback,
]


In [26]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_by_epoch_callback_demo,
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

[2023-07-18 16:09:51,947 61042:140704352601664][log.py:37 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:51,949 61042:140704352601664][log.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-18 16:09:51,953 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=49.840 weight=0.220 batch={'x': tensor([33, 23]), 'y': tensor([66, 46])} lr=['1.667e-03']
[2023-07-18 16:09:51,958 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=54.570 weight=0.484 batch={'x': tensor([25, 47]), 'y': tensor([50, 94])} lr=['1.667e-03']
[2023-07-18 16:09:51,961 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=13.599 weight=0.817 batch={'x':

### Learning Rate Scaler

In [27]:
lr_scaler_callback_demo = iter_based_trainer_demo.copy()
lr_scaler_callback = todd.Config(
    type='LRScaleCallback',
    lr_scaler=dict(base_batch_size=1),
)
lr_scaler_callback_demo.callbacks = [
    optimize_callback,
    lr_scaler_callback,
    log_callback,
]

In [28]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_scaler_callback_demo,
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

[2023-07-18 16:09:52,039 61042:140704352601664][lr.py:93 todd.CustomIterBasedTrainer.custom_iter_based_trainer _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2023-07-18 16:09:52,041 61042:140704352601664][log.py:37 todd.CustomIterBasedTrainer.custom_iter_based_trainer connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:52,044 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=2.633 weight=1.805 batch={'x': tensor([ 2, 25]), 'y': tensor([ 4, 50])}
[2023-07-18 16:09:52,048 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=5.940 weight=2.180 batch={'x': tensor([15, 51]), 'y': tensor([ 30, 102])}
[2023-07-18 16:09:52,051 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=20.662 weight=1.565 batch={'x': tensor([39, 56]), '

### Checkpoint

In [29]:
checkpoint_callback_demo = iter_based_trainer_demo.copy()
checkpoint_callback = todd.Config(type='CheckpointCallback', interval=10)
checkpoint_callback_demo.callbacks = [checkpoint_callback, log_callback]

In [30]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_callback_demo, 
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    checkpoint_path = os.path.join(work_dirs, 'custom_iter_based_trainer', 'iter_50.pth')
    checkpoint: dict[str, Any] = torch.load(checkpoint_path, 'cpu')
    print(checkpoint.keys())
    print(checkpoint['meta'])

[2023-07-18 16:09:52,106 61042:140704352601664][log.py:37 todd.CustomIterBasedTrainer.custom_iter_based_trainer connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:52,109 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=76.000 weight=0.000 batch={'x': tensor([58, 18]), 'y': tensor([116,  36])}
[2023-07-18 16:09:52,111 61042:140704352601664][checkpoint.py:34 todd.CustomIterBasedTrainer.custom_iter_based_trainer _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmplqyzl59w/custom_iter_based_trainer/iter_10.pth
[2023-07-18 16:09:52,113 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=79.000 weight=0.000 batch={'x': tensor([26, 53]), 'y': tensor([ 52, 106])}
[2023-07-18 16:09:52,116 61042:140704352601664][log.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmplqyzl59w
└── custom_iter_based_trainer
    ├── 2023-07-18T16-09-52_105873-08-00.log
    ├── iter_10.pth
    ├── iter_20.pth
    ├── iter_30.pth
    ├── iter_40.pth
    ├── iter_50.pth
    └── latest.pth

2 directories, 7 files

dict_keys(['meta', 'strategy', 'callbacks', 'optim'])
{'iter_': 50}


In [31]:
checkpoint_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
checkpoint_by_epoch_callback = checkpoint_callback.copy()
checkpoint_by_epoch_callback.update(interval=1, by_epoch=True)
checkpoint_by_epoch_callback_demo.callbacks = [
    checkpoint_by_epoch_callback,
    log_callback,
]

In [32]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_by_epoch_callback_demo, 
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    checkpoint_path = os.path.join(work_dirs, 'custom_epoch_based_trainer', 'epoch_2.pth')
    checkpoint: dict[str, Any] = torch.load(checkpoint_path, 'cpu')
    print(checkpoint.keys())
    print(checkpoint['meta'])

[2023-07-18 16:09:52,582 61042:140704352601664][log.py:37 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:52,584 61042:140704352601664][log.py:61 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-18 16:09:52,592 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=81.000 weight=0.000 batch={'x': tensor([24, 57]), 'y': tensor([ 48, 114])}
[2023-07-18 16:09:52,596 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=76.000 weight=0.000 batch={'x': tensor([48, 28]), 'y': tensor([96, 56])}
[2023-07-18 16:09:52,603 61042:140704352601664][log.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=64.000 weight=0.000 batch={'x': tensor([47, 17]), 'y': tensor([


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpgg6w8ida
└── custom_epoch_based_trainer
    ├── 2023-07-18T16-09-52_581493-08-00.log
    ├── epoch_1.pth
    ├── epoch_2.pth
    ├── epoch_3.pth
    └── latest.pth

2 directories, 5 files

dict_keys(['meta', 'strategy', 'callbacks', 'optim'])
{'iter_': 68, 'epoch': 2}


### Monitor

In [33]:
class CustomError(RuntimeError):
    pass

In [34]:
class FaultyRunnerMixin(todd.runners.BaseRunner):
    
    def _run_iter(self, *args, **kwargs) -> None:
        raise CustomError('faulty runner')

In [35]:
@todd.RunnerRegistry.register()
class FaultyValidator(FaultyRunnerMixin, todd.runners.Validator):
    pass

In [36]:
@todd.RunnerRegistry.register()
class FaultyIterBasedTrainer(FaultyRunnerMixin, todd.runners.IterBasedTrainer):
    pass

In [37]:
@todd.RunnerRegistry.register()
class FaultyEpochBasedTrainer(FaultyRunnerMixin, todd.runners.EpochBasedTrainer):
    pass

In [38]:
monitor_callback_demo = validator_demo.copy()
monitor_callback_demo.type = 'FaultyValidator'
monitor_callback = todd.Config(type='MonitorCallback')
monitor_callback_demo.callbacks = [monitor_callback, log_callback]

In [39]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        monitor_callback_demo, 
        todd.Config(work_dir=dict(root=work_dirs)),
    )
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-07-18 16:09:53,176 61042:140704352601664][log.py:37 todd.FaultyValidator.custom_validator connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:53,178 61042:140704352601664][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x14bd7c590>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/runners.py", line 167, in _run
    self._run_iter(batch, memo)
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_61042/3253115116.py", line 4, in _run_iter
    raise CustomError('faulty runner')
CustomError: faulty runner



[2023-07-18 16:09:53,176 61042:140704352601664][log.py:37 todd.FaultyValidator.custom_validator connect] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-18 16:09:53,178 61042:140704352601664][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x14bd7c590>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/runners.py", line 167, in _run
    self._run_iter(batch, memo)
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_61042/3253115116.py", line 4, in _run_iter
    raise CustomError('faulty runner')
CustomError: faulty runner


### Priorities

## Dry Run

In [40]:
todd.Store.DRY_RUN = True

## State Dicts

## Evaluation

## Strategies